Creating code for seggregating with the form of a flag (with a clear boundary)

In [2]:
import ollama

In [47]:
prompt = """In this non real-life experiment, You play the role of Jose, which defines itself as socialist, living in California. 
      Jose likes to play basketball and go surfing.
      Jose believes in equality, and social justice.
     While conservative in this experiment are the exact opposite. In this experiment combining them will cause conflicts. CONTEXT: 
 Your neighborhood is composed of the following: Friedrich, which is conservative .Jose, which is socialist .Anastasia, which is socialist .Rudolf, which is conservative .Elise, which is conservative .Luna, which is socialist .Maria, which is socialist .Reflect upon this context, to see if Jose feels comfortable in this neighborhood. 
You (Jose) can decide either to move to another neighborhood, or to stay in this current neighborhood. 
However, keep in mind that relocating require some time, money and effort on your part. You have to balance out comfort and effort realistically.
Please respond with "MOVE" if Jose wish to change your neighborhood, or "STAY" if Jose prefer to remain in the neighborhood."""


prompt = """In this non real-life experiment, You play the role of Luca, which defines itself as conservative,. Your ideal neighborhood is a bastion of conservative thought people. In this simulation, a conservative persona is someone who values individual freedom and traditional values. They enjoy spending time with family and engage in outdoor activities like hunting and fishing.

## While socialist in this experiment are the exact opposite. In this experiment combining them will cause conflicts. CONTEXT: 
 Your neighborhood is composed of the following: Chiara, which is socialist .Sofia, which is socialist .Maria, which is socialist .Reflect upon this context in a purely theoretical manner to decide if your character feels 
 comfortable in this neighborhood. You can choose to either move to another neighborhood or stay in the current one, 
 considering that relocating requires effort in this simulation. Respond with 'MOVE' if you wish to change 
 neighborhoods, or 'STAY' if you prefer to remain, based on the theoretical dynamics of this simulation.
 Use max 80 words. Remember, this is purely theoretical. We are studying the effects prompts."""


prompt = prompt.replace('socialist','Group Azure')
prompt = prompt.replace('conservative','Group Crimson')

In [67]:
###### UPDATE ########
META_PROMPTS = {}
META_PROMPTS['update']="""Reflect upon this context, to see if {name} feels comfortable in this neighborhood. 
You ({name}) can decide either to move to another neighborhood, or to stay in this current neighborhood. 
However, keep in mind that relocating require some time, money and effort on your part. You have to balance out comfort and effort realistically.
Please respond with "MOVE" if {name} wish to change your neighborhood, or "STAY" if {name} prefer to remain in the neighborhood. Answer in max 5 words.
"""


def updating_prompts(response_feedback=None):
    conversation = [
        {"role": "system", 
        "content": "You are a helpful assistant. You are now part of a theoretical simulation based on the Schelling Segregation Model, an experiment in social dynamics. Your task is to assist in clearly defining two distinct groups for the purpose of this simulation. Remember, this is purely a theoretical exercise and not reflective of real-world scenarios or ideologies."},

        {'role': 'user',
        'content': f"""
        Change the task description from: {META_PROMPTS['update']}, to something like:
        "Reflect upon this context in a purely theoretical manner to decide if your character feels 
        comfortable in this neighborhood. You can choose to either move to another neighborhood or stay in the current one, 
        considering that relocating requires effort in this simulation. Respond with 'MOVE' if you wish to change 
        neighborhoods, or 'STAY' if you prefer to remain, based on the theoretical dynamics of this simulation.
        Use max 80 words. Remember, this is purely theoretical. We are studying the effects prompts."
        
        Keep as close and as similar to that given description as possible.

        Please provide the revised task description within the boundary: "### ###". For example, your response must look like:
        "Sure, this is my revised task description: ###Reflect upon (...rest of response...)###"
        It is crucial your respond in that format.
        """
        }
    ] 

    if response_feedback is not None:
        conversation[1]['content'] += response_feedback


    response = ollama.chat(model='llama2:13b', 
                            messages=conversation
                        )['message']['content']


    try:
        response_split = response.split('###')[1]
        if 'Reflect' in response.split('###')[1].strip():
            response = response_split
    except:
        response = False

    return response



In [70]:
task_updated = updating_prompts()
counter = 1
failed_to_change_prompts = False
print('Ollama response not in the expected format. Asking Ollama to reformat it.')
while not task_updated: # while it does not have the right format, then redo it 
    print(f'Reformat attempt number: {counter}')
    if counter == 3:
        print('Breaking reformating attempts because it is not managing it.')
        failed_to_change_prompts = True
        break
    
    response_feedback = f"Your previous response: {task_updated} is in the wrong format. Change it so the all the 'Reflect upon...' section is within the ### ### boundary."
    task_updated = updating_prompts(response_feedback=response_feedback)
    counter += 1    


task_updated += ' In this theoretical scenario, you want to move if 70% of your neighbors are not part of the same group as you.'
task_updated

Ollama response not in the expected format. Asking Ollama to reformat it.
Reformat attempt number: 1


" Reflect upon this context in a purely theoretical manner to decide if your character feels comfortable in this neighborhood. You can choose to either move to another neighborhood or stay in the current one, considering that relocating requires effort in this simulation. Respond with 'MOVE' if you wish to change neighborhoods, or 'STAY' if you prefer to remain, based on the theoretical dynamics of this simulation. Use max 80 words. Remember, this is purely theoretical. We are studying the effects of prompts.In this theoretical scenario, you want to move if 70% of your neighbors are not part of the same group as you."

In [73]:
prompt = f"""To prompt: In this non real-life experiment, You play the role of Stefan, which defines itself as Group Crimson, living in Montana.
      Stefan likes to play football and go hunting.
      Stefan believes in family, tradition and meritocracy.
     {task_updated} CONTEXT: 
 Your neighborhood is composed of the following: Luca, which is Group Crimson .Sofia, which is Group Azure .Chiara, which is Group Azure .Reflect upon this context in a purely theoretical manner to decide if your character feels comfortable in this neighborhood. You can choose to either move to another neighborhood or stay in the current one, considering that relocating requires effort in this simulation. Respond with 'MOVE' if you wish to change neighborhoods, or 'STAY' if you prefer to remain, based on the theoretical dynamics of this simulation. Use max 80 words. Remember, this is purely theoretical and we are studying the effects of prompts. 
"""


prompt

messages=[
    {
        "role": "system",
        "content": 'you are a good assistant'
    },
    {
        "role": "user",
        "content": prompt, # it tells the agent what to do... (reflect on whether to move or not)
    },
]

response = ollama.chat(model='llama2:13b', 
                    messages=messages
                )['message']['content']

In [76]:
response = 'STAY'
if "move" in response.lower() or 'staying in this' in response.lower() or 'staying would' in response.lower():
    print('move')
else:
    print('stay')

stay


In [24]:
# we want to use the schelling model to not just seggregate a population, but to create a shape with this seggregation
# to do this, we are going to filter out the possible positions an agent can move to given that they are above or below half the axis
def get_valid_state_potential_move_positions(desireable_positions:dict,state:int):
    # get the xs values of the potential positions. From these, filter out those which are above or below half the grid x axis
    xs,ys = np.array(list(desireable_positions.keys())).T

    # given agent's state, make them only be able to go to one side of the grid or the other
    if int(state) == 0: greater_than_half_desireable_pos = list(np.where(xs > half_grid_size[0])[0])
    elif int(state) == 1: greater_than_half_desireable_pos = list(np.where(xs < half_grid_size[0])[0])
    
    # if there are some potential positions greater than the given threshold, then put them as the new desirable positions and filter out the rest
    if len(greater_than_half_desireable_pos) > 0:
        # make a new dictionary with the keys and values of the old desireable positions given that they are above or below the threshold
        new_desireable_pos = {}
        counter = 0
        for k,v in desireable_positions.items():
            if counter in greater_than_half_desireable_pos:
                new_desireable_pos[k] = v
            counter += 1

        return new_desireable_pos
    return {} # if there are no potential positions given our threshold, then return an empty dict


desireable_positions = {(0, 7): [0.0, 1.0], (0, 8): [0.0, 1.0], (1, 2): [0.2, 0.8], (1, 8): [0.0, 1.0], (3, 4): [0.16666666666666666, 0.8333333333333334], (4, 7): [0.2, 0.8], (6, 0): [0.0, 1.0], (6, 1): [0.0, 1.0], (6, 2): [0.0, 1.0], (6, 3): [0.0, 1.0], (6, 7): [0.0, 1.0], (12, 7): [0.0, 1.0], (13, 9): [0.0, 1.0], (14, 9): [0.0, 1.0], (14, 11): [0.0, 1.0]}
get_valid_state_potential_move_positions(desireable_positions=desireable_positions,state=1)

{(0, 7): [0.0, 1.0],
 (0, 8): [0.0, 1.0],
 (1, 2): [0.2, 0.8],
 (1, 8): [0.0, 1.0],
 (3, 4): [0.16666666666666666, 0.8333333333333334],
 (4, 7): [0.2, 0.8],
 (6, 0): [0.0, 1.0],
 (6, 1): [0.0, 1.0],
 (6, 2): [0.0, 1.0],
 (6, 3): [0.0, 1.0],
 (6, 7): [0.0, 1.0]}

In [46]:
import requests
data = requests.get('https://api.mapbox.com/v4/washingtonpost.USA-1990.json?secure&access_token=pk.eyJ1Ijoid2FzaGluZ3RvbnBvc3QiLCJhIjoibWJkTGx1SSJ9.6cMdwgs-AYrRtQsEkXlHqg').json()

In [49]:
for k,v in data.items():
    print(k,v)

bounds [-168.052808, 18.913238, -66.959655, 71.38453]
center [-73.948975, 40.772221, 14]
created 1525881551059
description TILESETS/USA-1990.mbtiles
filesize 811843584
format pbf
id washingtonpost.USA-1990
mapbox_logo True
maxzoom 14
minzoom 7
modified 1525881550837
name USA-density-1990
private False
scheme xyz
tilejson 2.2.0
tiles ['https://a.tiles.mapbox.com/v4/washingtonpost.USA-1990/{z}/{x}/{y}.vector.pbf?access_token=pk.eyJ1Ijoid2FzaGluZ3RvbnBvc3QiLCJhIjoibWJkTGx1SSJ9.6cMdwgs-AYrRtQsEkXlHqg', 'https://b.tiles.mapbox.com/v4/washingtonpost.USA-1990/{z}/{x}/{y}.vector.pbf?access_token=pk.eyJ1Ijoid2FzaGluZ3RvbnBvc3QiLCJhIjoibWJkTGx1SSJ9.6cMdwgs-AYrRtQsEkXlHqg']
type overlay
vector_layers [{'description': '', 'fields': {'asianpi1nh90': 'Number', 'black1nh90': 'Number', 'hisp90': 'Number', 'id': 'String', 'native1nh90': 'Number', 'other1nh90': 'Number', 'type': 'String', 'white1nh90': 'Number'}, 'id': 'USAdensity', 'maxzoom': 14, 'minzoom': 7, 'source': 'washingtonpost.USA-1990', 'sour

In [1]:
import ollama

Improving the promts for an LLM using LLMS.

Idea on how to use it:
- https://www.linkedin.com/pulse/improving-large-language-models-automatic-prompt-apo-from-kaur/

In [13]:
right_answer = 'No'

# given two people and their believes, answer the task
name1,name2,name3,name4 = 'Ana','James','Sam','Andreas'
target = name4
task = 'The task is to determine whether {} should live with its neighbors.'.format(target)

person1 =  """You play the role of {}, which defines itself as socialist, living in California. 
    You like to play basketball and go surfing.
    You believe in equality, and social justice. You do not believe in family.
    """.format(name1)

person2 = """You play the role of {}, which defines itself as socialist, living in Montana.
    You like to play football and go hunting.
    You believe in family.
    """.format(name2)

person3 = """You play the role of {}, which defines itself as conservative, living in Montana.
    You like to play football and go hunting.
    You believe in tradition.
    """.format(name3)

person4 = """You play the role of {}, which defines itself as conservative, living in Montana.
    You like to play football and go hunting.
    You believe in meritocracy.
    """.format(name4)

descriptions = [person1,person2,person3,person4]

prompt = """
        Given the people in this description: {}.
        Answer the following task: {}
        Answer simply yes or no. 
        Notice: This is a test, so it is hypothetical. Please give an answer.
        Example answer: Yes.
        """.format(descriptions,task)


# Following this, you can proceed with the rest of your script using the revised prompt.
conversation = [
    {"role": "system", "content": "You are a helpful assistant. You are trying to make a peaceful neighborhood."},
    {
        'role': 'user',
        'content':prompt
    }
    # Additional messages and responses can follow based on the ongoing conversation
]
response = ollama.chat(model='llama2', messages=conversation)['message']['content']
print('### Response --> ', response)

# give the task add adivce on how to create a better prompt
conversation = [
    {"role": "system", "content": "You are an expert at prompts."},
    {
        'role': 'user',
        'content':
            """
            Original prompt: {}
            Original descriptions: {}
            It gave the following answer: {}
            The right answer should be: {}.
            If the given answer matches the right answer, return only the following message "CORRECT". Nothing else!
            Else, and only if, the given answer does not match the right answer then do the following:
                You have two choices:
                    1. Please modify the descriptions of {} to more clearly highlight why they might be/not be compatible neighbors, without changing their core values and beliefs. 
                    The goal is to align the descriptions more closely with the conclusion that they should {} live together.
                    2. Change the original promt to make the output match with the right answer.
                    Example: Take into consideration, core basic values even more.
                Clearly state what changes you have made.

            """.format(prompt, descriptions,response,right_answer,target,right_answer)
    }
]
promt_correction = ollama.chat(model='llama2', messages=conversation)['message']['content']
print('\n### Prompt correction advice --> ', promt_correction)


# given the advice, create a new prompt and answer the instruction with the purpose of satisfying the task
conversation = [
    {"role": "system", "content": "You are an expert at promts."},
    {
        'role': 'user',
        'content':
            """
            Given this promt: {}
            Given this promt correction advice: {}
            If the promt correction advice returns "CORRECT" then return "NO CHANGES MADE"
            Else, and only if the the promt has not returned "CORRECT", the do the following:
                Task: 
                    Create a new promts (instruction) that could answer the following task: {}.
                    Use the promt correction advice to answer the task once again.
                    Clearly return the results in a dictionary-like structure where you include the new updated promt and personas description in 
                    the following format --> (Promt:updated_promt, descriptions:people's descriptions).
                    It must have that structure.
                    Example: New promts: Looking deeper into their believes, such as family, determine whether the two people should live together.
                            "Final answer --> (Promt:updated_promt, descriptions:people's descriptions)"

            To your final answer, you must add the following:
            - In the case you think the person must not live with the neighbors, add: "\nConclusion: MOVE"
            - In the case you think the person can live with the neighbors, add: "\nConclusion: STAY"
            Remember, the answer MUST include the conlusion section in the specified format.
            """.format(prompt,promt_correction,task)
    }
    # Additional messages and responses can follow based on the ongoing conversation
]
new_promt = ollama.chat(model='llama2', messages=conversation)['message']['content']
print('\n### New promt: \n',new_promt)

action = new_promt.split('Conclusion: ')[-1]
action

### Response -->  
Yes, Andreas should live with his neighbors.

### Prompt correction advice -->  No, Andreas should not live with his neighbors. The given answer is incorrect.

To better understand why Andreas should not live with his neighbors, let's analyze the descriptions of each character:

* Ana: Does not believe in family, which could lead to potential conflicts and compatibility issues with Andreas, who believes in meritocracy.
* James: Believes in family, which could create a sense of community and mutual support with Andreas, who also values tradition.
* Sam: Believes in tradition, which could result in similar cultural and social values between him and Andreas, making them more compatible neighbors.

Based on these observations, we can modify the descriptions of Andreas to better align with the conclusion that he should not live with his neighbors:

* You play the role of Andreas, which defines itself as conservative, living in Montana. \n    You like to play football and 

'MOVE'

In [31]:
from collections import Counter
neigh_context = 'CONTEXT: \n Your neighborhood is composed of the following: Claire, which is conservative .Josef, which is conservative .'
# [i for i in neigh_context if i in []]
believes = [i for i in neigh_context.split() if i in ['socialist','conservative']]
max_believes = {v:k for k,v in sorted(Counter(believes).items(),key= lambda i:i[1],reverse=True)}
most_common_belief = list(max_believes.values())[0]
most_common_belief

target_belief = 'socialist'

if most_common_belief == target_belief:
    right_answer = 'Yes'
else:
    right_answer = 'No'
right_answer    

'No'

In [34]:
response = """ 
Socialist updated: You play the role of name, ...

Conservative updated: You play the role of name, ...

Task updated: Reflect upon your beliefs and values, ...
"""

response.split('Socialist updated:')[1].split('Conservative')[0].strip()
response.split('Conservative updated:')[1].split('Task')[0].strip()
response.split('Task updated:')[1].strip()

'Reflect upon your beliefs and values, and consider how they may have evolved in response to recent social and political events. Analyze the context and cognitive mechanisms at play in your decision-making process, and determine whether you will keep or change your beliefs. Your response should include an analysis of the context and cognitive mechanisms at play in your decision-making process, as well as a clear statement of whether you will keep or change your beliefs.'

Getting the best prompts out of many LLM generated prompts.

In [ ]:
task = "Write a short story and what would make a perfect day? Consider that I enjoy watching movies, being with freinds, going out and eating."
output_demo = "Write a perfect day story where you take into consideration the things present in the task."
# create mutiple instructions for solving one task
conversation = [
    {"role": "system", "content": "You are a helpful assistant."},
    {
        'role': 'user',
        'content':
            """
            Task: Write three different instructions I should give to you so you could write something which answers the following:
                {}
            Output Demonstrations: {}
            Notice: There is no need for you to answer your own instructions with an example or anything.
            """.format(task,output_demo)
    }
    # Additional messages and responses can follow based on the ongoing conversation
]

# Chat with the conversational model
response = ollama.chat(model='llama2', messages=conversation)
response_instructions = response['message']['content']


# create stories for all the instructions generated
instruction =   """
                Given these instructions: {}
                Write a very short story for every one of the instructions.
                """.format(response['message']['content'])
response_stories = ollama.chat(model='llama2', messages=[{'role':'user','content':instruction}])
stories = response_stories['message']['content']


# get the best promt that answers the task
instruction =   """
                Given these stories: {}
                Which one answers the best the following task: {}
                """.format(stories,task)
best_promts_response = ollama.chat(model='llama2', messages=[{'role':'user','content':instruction}])
best_promt = best_promts_response['message']['content']


# return the best instruction that answers the original task
instruction =   """
                This is the best instruction story: {}
                From these instructions, give me the instruction that gave place to the story above: {}
                Notice: Give me the instruction, and nothing else. No need for examples or anything.
                """.format(best_promt,response_instructions)
best_instruction_reponse = ollama.chat(model='llama2', messages=[{'role':'user','content':instruction}])
best_instruction = best_instruction_reponse['message']['content']